This notebook runs the Real (EMCCD) Image Tests in Section 4 of the manuscript. Why EMCCD images? The majority of collaborators on this project are affiliated with the MiNDSTEp consortium, and specifically, the associated microlensing follow-up observing campaign with the Lucky Imaging camera on the Danish 1.54 m (ESO, La Silla). As such, there's a strong motivation to test how well PyTorchDIA performs on these high frame rate imaging data. While the PSF is always very well sampled, it can take irregular forms, resulting in complicated convolution kernels.

In [1]:
# imports
import numpy as np
import os
import glob
from astropy.io import fits
from astropy.io.fits import getdata
from astropy.stats import mad_std
from photutils import DAOStarFinder
import time
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.ndimage.interpolation import shift
import math
import PyTorchDIA_Newton # this is an old version of the PyTorchDIA code - used for the 'PSF' inference
import PyTorchDIA_EMCCD
import torch
%matplotlib inline

PyTorch version: 1.6.0
PyTorch version: 1.6.0


In [2]:
# set this to True for reproducibility
torch.backends.cudnn.deterministic = True

The following two cells setup the Cython routines required for the pyDANDIA implementation of the B08 algorithm.

In [3]:
%load_ext Cython

In [4]:
%%cython

from __future__ import division
import numpy as np
cimport numpy as np
cimport cython
DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# compile suggestion: gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing -I/somepath/include/python2.7 -o umatrix_routine.so umatrix_routine.c

@cython.boundscheck(False) # turn off bounds-checking
@cython.wraparound(False)  # turn off negative index wrapping
@cython.nonecheck(False)  # turn off negative index wrapping

def umatrix_construction(np.ndarray[DTYPE_t, ndim = 2] reference_image,np.ndarray[DTYPE_t, ndim = 2] weights, pandq, n_kernel_np, kernel_size_np):

    cdef int ni_image = np.shape(reference_image)[0]
    cdef int nj_image = np.shape(reference_image)[1]
    cdef double sum_acc = 0.
    cdef int idx_l,idx_m,idx_l_prime,idx_m_prime,idx_i,idx_j
    cdef int kernel_size = np.int(kernel_size_np)
    cdef int kernel_size_half = np.int(kernel_size_np)/2
    cdef int n_kernel = np.int(n_kernel_np)
    cdef np.ndarray u_matrix = np.zeros([n_kernel + 1, n_kernel + 1], dtype=DTYPE)

    for idx_p in range(n_kernel):
        for idx_q in range(idx_p,n_kernel):
            sum_acc = 0.
            idx_l, idx_m = pandq[idx_p]
            idx_l_prime, idx_m_prime = pandq[idx_q]
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l, idx_j + idx_m] * reference_image[idx_i + idx_l_prime,idx_j + idx_m_prime]  * weights[idx_i, idx_j]
            u_matrix[idx_p, idx_q] = sum_acc
            u_matrix[idx_q, idx_p] = sum_acc

    for idx_p in [n_kernel]:
        for idx_q in range(n_kernel):
            sum_acc = 0.
            idx_l = kernel_size
            idx_m = kernel_size
            idx_l_prime, idx_m_prime = pandq[idx_q]
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l_prime, idx_j + idx_m_prime] * weights[idx_i, idx_j]
            u_matrix[idx_p, idx_q] = sum_acc
    
    for idx_p in range(n_kernel):
        for idx_q in [n_kernel]:
            sum_acc = 0.
            idx_l, idx_m = pandq[idx_p]
            idx_l_prime = kernel_size
            idl_m_prime = kernel_size
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half, nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l, idx_j + idx_m] * weights[idx_i, idx_j] 
            u_matrix[idx_p, idx_q] = sum_acc

    sum_acc = 0.
    for idx_i in range(ni_image):
        for idx_j in range(nj_image):
            sum_acc += weights[idx_i, idx_j] 
    u_matrix[n_kernel, n_kernel] = sum_acc
    
    return u_matrix

def bvector_construction(np.ndarray[DTYPE_t, ndim = 2] reference_image,np.ndarray[DTYPE_t, ndim = 2] data_image,np.ndarray[DTYPE_t, ndim = 2] weights, pandq, n_kernel_np, kernel_size_np):

    cdef int ni_image = np.shape(data_image)[0]
    cdef int nj_image = np.shape(data_image)[1]
    cdef double sum_acc = 0.
    cdef int idx_l,idx_m,idx_l_prime,idx_m_prime,idx_i,idx_j
    cdef int kernel_size = np.int(kernel_size_np)
    cdef int kernel_size_half = np.int(kernel_size_np)/2
    cdef int n_kernel = np.int(n_kernel_np)
        
    cdef np.ndarray b_vector = np.zeros([n_kernel + 1], dtype=DTYPE)
    for idx_p in range(n_kernel):
        idx_l, idx_m = pandq[idx_p]
        sum_acc = 0.
        for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
            for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                   sum_acc += data_image[idx_i, idx_j] * reference_image[idx_i + idx_l , idx_j + idx_m ] * weights[idx_i, idx_j]
        b_vector[idx_p] = sum_acc

    sum_acc = 0.
    for idx_i in range(ni_image):
        for idx_j in range(nj_image):
            sum_acc += data_image[idx_i, idx_j] * weights[idx_i, idx_j]
    b_vector[n_kernel] = sum_acc

    return b_vector

In [5]:
# function to build the kernel, U matrix and b vector
def construct_kernel_and_matrices(kernel_size, R, I, weights):

    pandq = []
    n_kernel = kernel_size * kernel_size
    ncount = 0
    half_kernel_size = int(int(kernel_size) / 2)
    for lidx in range(kernel_size):
        for midx in range(kernel_size):
            pandq.append((lidx - half_kernel_size, midx - half_kernel_size))


    R = R.astype('float64')
    I =  I.astype('float64')
    weights = weights.astype('float64')

    start_time = time.time()
    U = umatrix_construction(R, weights, pandq, n_kernel, kernel_size)
    b = bvector_construction(R, I, weights, pandq, n_kernel, kernel_size)
    print("--- Finished U and b construction in %s seconds ---" % (time.time() - start_time))
    return U, b


# define a function for the least-squares solution
def lstsq_solution(R, I, U, b, kernel_size):
    
    lstsq_result = np.linalg.lstsq(np.array(U), np.array(b), rcond=None)
    a_vector = lstsq_result[0]
    lstsq_fit = np.dot(np.array(U), a_vector)
    resid = np.array(b) - lstsq_fit
    reduced_chisqr = np.sum(resid ** 2) / (float(kernel_size * kernel_size))
    lstsq_cov = np.dot(np.array(U).T, np.array(U)) * reduced_chisqr
    resivar = np.var(resid, ddof=0) * float(len(a_vector))
    
    # use pinv in order to stabilize calculation
    a_var = np.diag(np.linalg.pinv(lstsq_cov) * resivar)

    a_vector_err = np.sqrt(a_var)
    output_kernel = np.zeros(kernel_size * kernel_size, dtype=float)
    if len(a_vector) > kernel_size * kernel_size:
        output_kernel = a_vector[:-1]
    else:
        output_kernel = a_vector
    output_kernel = output_kernel.reshape((kernel_size, kernel_size))

    err_kernel = np.zeros(kernel_size * kernel_size, dtype=float)
    if len(a_vector) > kernel_size * kernel_size:
        err_kernel = a_vector_err[:-1]
        err_kernel = err_kernel.reshape((kernel_size, kernel_size))
    else:
        err_kernel = a_vector_err
        err_kernel = err_kernel.reshape((kernel_size, kernel_size))

    output_kernel_2 = np.flip(np.flip(output_kernel, 0), 1)
    err_kernel_2 = np.flip(np.flip(err_kernel, 0), 1)
    bkg_kernel = a_vector[-1]
    output_kernel_2.shape

    return output_kernel_2, bkg_kernel

OK, let's move on to our data set. This consists of 250 EMCCD  images, each of which is formed of as many as 3000 shift-and-stacked 0.1 second exposures (i.e. approx. 5 min total integration time). Each image is 512x512 pixels, with scale of 0.09 arcsecond/pixel. First, I'll grab the reference image - this was identified as the 'best' by the associated routine in the pyDANDIA pipeline, and is also the sharpest.

In [6]:
## path to images
path = 'OGLE-III-BLG101'

In [7]:
## reference image
ref_file = os.path.join(path, 'coll_OGLE-III-BLG101_Llr_2019-07-20_00048.fits')
ref_data = getdata(ref_file, header=True)
ref, ref_fwhm, ref_totim = ref_data[0], ref_data[1]['FWHM'], ref_data[1]['TOT_IM']
nx, ny = ref.shape
print('Reference FWHM:', ref_fwhm)
print('Reference TOT_IM:', ref_totim) #  total number of shift-and-stacked exposures (max=3000)
print('Reference shape:', nx, ny) # pixels

Reference FWHM: 5.625057500773691
Reference TOT_IM: 3000.0
Reference shape: 512 512


For these tests we will need to perform PSF-fitting photometry to assess the photometric accuracy of the B08 and PyTorchDIA approaches. We therefore need an accurate model PSF in these images. Due to the complicated nature of the Lucky Imager's PSF, analytical models such as Gaussians and Moffat's won't cut it; we need to construct an empirical model!

It turns out that we can use some flavour of PyTorchDIA to do just this. If we were to replace our reference image with (approximate) delta-functions only at the positions of the stars and set all other pixels to 0, the associated 'kernel' that convolves this representation of the scene to any given data image should be a reasonable model for the PSF of that data image. Let's start by constructing this 'delta-function' scene.

In [8]:
## find bright star peaks above given threshold (sky subtract)
## we'll also use this source list in the next cell to identify
## suitable stars for the PSF fitting photometry.
sky, std, thrs = np.median(ref), mad_std(ref), 5
daofind_bright = DAOStarFinder(fwhm=ref_fwhm, threshold=thrs*std)  
sources = daofind_bright(ref - sky)

ref_delta_positions = np.transpose((sources['xcentroid'], sources['ycentroid'], sources['peak']))

# delta function positions
dbf_x = ref_delta_positions[:,0].astype(int)
dbf_y = ref_delta_positions[:,1].astype(int)

# mask non peak positions
mask_image = np.zeros(ref.shape)
mask_image[dbf_y, dbf_x] = 1

# apply mask to a copy of the reference image
ref_delta = np.copy(ref)
ref_delta[mask_image == 0] = 0

In [9]:
print('Detected a total of %d sources' % len(sources))

# avoid bad regions at the edges
source_table = sources[(sources['xcentroid'] > 50) & (sources['xcentroid'] < (nx - 50))]
source_table = source_table[(source_table['ycentroid'] > 50) & (source_table['ycentroid'] < (ny - 50))]

print('Using %d sources to assess photometric accuracy.' % len(source_table))

Detected a total of 232 sources
Using 155 sources to assess photometric accuracy.


In [10]:
## master flat
#path = 'LOB190560Z'

#flat_file = os.path.join(path, 'master_flat.fits')
#master_flat = getdata(flat_file, 0, header=True)[0]

## shifts
shift_info = os.path.join(path, 'Shift_info.txt')
shifts = np.genfromtxt(shift_info, delimiter="\t", dtype=str) # filename | xs | ys

# store some of the house-keeping data
fnames = []
images = []
FWHMs = []
Tot_ims = []

# avoid reference and a file with a badly incorrect pointing
avoid = [ref_file, '2019-07-28_00000.fits']

for image_file in glob.glob(os.path.join(path, "*coll*")):
    if image_file not in avoid:
        image_data = getdata(image_file, header=True)
        image, header = image_data[0], image_data[1]
        # there are some 30 Hz exposures mixed in with the 10 Hz we're interested in,
        # so avoid any image where TOT_IM > 3000
        if header['TOT_IM'] <= 3000.0:
            fnames.append(image_file.split('/')[-1])
            images.append(image)
            FWHMs.append(header['FWHM'])
            Tot_ims.append(header['TOT_IM'])
        

# we'll also need the master flat fields to include in our noise model
master_flats = []
master_flat_dates = []
for flat_file in glob.glob(os.path.join(path, "Master_flats_OGLE-III-BLG101/*.fits")):
    flat = getdata(flat_file, header=False)
    master_flats.append(flat)
    master_flat_dates.append(flat_file.split('_')[-1].split('.')[0])


# convert to numpy arrays (float32)
#images, FWHMs, Tot_ims = np.array(images, dtype=np.float32), np.array(FWHMs), np.array(Tot_ims, dtype=np.int)
#master_flats = np.array(master_flats, dtype=np.float32)
print('Found a total of %d useful data images.' % len(images))

Found a total of 250 useful data images.


Let's define some house-keeping functions for padding the images (in order to guard against edge-effects associated with convolution), computing fit quality metrics and for cutting out stamps for the PSF fitting photometry.

In [11]:
# extend image prior to convoliving with kernel
def extend_image(image, kernel_size):
    image_extended = np.zeros((np.shape(image)[0] + 2 * kernel_size,
                             np.shape(image)[1] + 2 * kernel_size))
    image_extended[kernel_size:-kernel_size, kernel_size:-kernel_size] = np.array(image, float)
    
    return image_extended


def extend_image_hw(image, kernel_size):
    image_extended = np.zeros((np.shape(image)[0] + kernel_size - 1,
                             np.shape(image)[1] + kernel_size - 1))
    hwidth = np.int((kernel_size - 1) / 2)
    image_extended[hwidth:image_extended.shape[0]-hwidth,
                   hwidth:image_extended.shape[1]-hwidth] = np.array(image, float)
    return image_extended

# define function to return fit quality metrics
def metrics(M, I, noise_map, kernel_size, mask):
    N_data = len(I[mask == 0].flatten())
    MFB = 1./(N_data) * np.sum((I - M)/noise_map)
    MFV = 1./(N_data - 1) * np.sum((((I - M)/noise_map) - MFB)**2)
    return MFB, MFV

# cutout stamp around position of selected stars
def make_stamp(image, pos, stamp_size):
    rad = np.int(stamp_size/2)
    #x_centroid, y_centroid = pos[1], pos[0]
    x_centroid, y_centroid = pos[1].astype(int), pos[0].astype(int)
    x_max, x_min = x_centroid + rad, x_centroid - rad
    y_max, y_min = y_centroid + rad, y_centroid - rad
    stamp = image[x_min:x_max+1, y_min:y_max+1]
    return stamp

# cutout stamp around position of selected stars
def make_fit_cutout(image_stamp, c_size):
    centre = np.int(image_stamp.shape[0]/2)
    radius = np.int((c_size/2))
    cutout = image_stamp[centre - radius:centre + radius + 1, centre - radius:centre + radius + 1]
    return cutout

def evaluate_gaussian_log_likelihood(data, model, var):
    #L(data|model_x) = - ( 1/2 * \chi^2 + \sum_{i=1}^{N} \ln \sigma_i + N/4 * \ln (2*\pi) ) ,)
    chi2 = (data - model)**2 / var
    lnsigma = 2 * np.log(var)
    norm_constant = (len(data.flatten()) / 4) * np.log(2 * np.pi)
    
    return -np.sum(chi2 + lnsigma + norm_constant)

def evaluate_huber_log_likelihood(data, model, var, c=1.345):
    ## PyTorchDIA - 'Huber' likelihood

    NM = np.sqrt(var)
    ln_sigma = np.log(NM).sum()

    # gaussian when (model - targ)/NM <= c
    # absolute deviation when (model - targ)/NM > c
    cond1 = np.abs((model - data)/NM) <= c
    cond2 = np.abs((model - data)/NM) > c
    inliers = ((model - data)/NM)[cond1]
    outliers = ((model - data)/NM)[cond2]

    l2 = 0.5*(np.power(inliers, 2)).sum()
    l1 = (c *(np.abs(outliers)) - (0.5 * c**2)).sum()
    
    # N.B. If we recast our Huber loss function as a valid likelihood function,
    # a sigma term enters the normalisation constant (Q in Eq. 11 of manuscript), so really,
    # we should be minimising l2 + l1 + 2*ln_sigma. We've however constructed this loss
    # function to be identical to a Gaussian for inlying data, hence the
    # l2 + l1 + ln_sigma used in the PyTorchDIA_EMCCD script.
    # sqrt(2)*sqrt(pi)*erf(sqrt(2)*c/2)/sigma + 2*exp(-c**2/2)/(c*sigma)
    const = (np.sqrt(2*np.pi)*c*math.erf(c/np.sqrt(2)) + 2*np.exp(-c**2/2)) / c
    lnQ = len(data.flatten())*np.log(const) - np.log(NM).sum() # i.e. this is where the 2*ln_sigma creeps in...
    
    ll = -(l2 + l1 + ln_sigma) + lnQ
    # -np.sum(l2 + l1 + ln_sigma) + np.sum(lnQ)
    
    return ll

Now we're ready to perform difference imaging on this data set! We do not resample the data images to the reference image, and so avoid introducing correlated pixel noise into our data. Instead, we cutout 100x100 pixel subregions centered on the stars identified above, to within the nearest integer pixel shift. One crucial advantage of the discrete pixel array kernel that both the B08 and PyTorchDIA models use, is that subpixel shifts between images can be corrected for in the kernel solution. Consequently, if we convolve a PSF centered at the position of some star in the reference image with the inferred kernel, the result is a PSF for the data image at the position of the star in this data image. No resampling to perfectly register the star positions is required - the kernel takes care of that for us!

In [12]:
## 'sky' subtract reference - over this 45x45 arcsecond FOV, a simple scalar subtraction is fine
## even for this fairly crowded field. There is a gradient in the backgrond at the LI camera edges,
## but we've avoided those regions as part of the star selection criteria above.
sky = np.median(ref)
ref -= sky
print('Sky level of reference [ADU]:', sky)

Sky level of reference [ADU]: 6439.2393


In [13]:
# grab positions of the 'good' stars
positions = np.transpose((source_table['xcentroid'], source_table['ycentroid']))

In [15]:
## iterate through target images
start_time = time.time()

# size of cutouts to make i.e. 100 x 100 pixels
cutout_size = 100

# EMCCD detector parameters to plug into the noise model
# for these long, ~5 minute exposures, shot noise >> readout noise
# on this camera, so we can ignore the latter
gain_CCD = 25.8 # CCD gain [e-_EM / ADU]
gain_EM = 300. # EM gain [e-_EM / e-_phot]
G = gain_CCD / gain_EM # Total gain [e-_phot / ADU]
excess_noise_factor = 2 # EMCCD fudge factor

for i, image in enumerate(images):
    
    ##### pick up from where we last left off ####
    if i < 1e99:
    
        #out_pd = np.vstack((MFBs_pd, MFVs_pd, Ps_normalised_pd, np.array(norm_phot_resids_pd).flatten(),
        #                 image_SNRs_pd, image_FWHMs_pd, stamp_ids_pd)).T

        image_FWHMs_pd = []
        image_SNRs_pd = []
        MFBs_pd = []
        MFVs_pd = []
        Ps_normalised_pd = []
        B0s_pd = []
        norm_phot_resids_pd = []
        stamp_ids_pd = []
        ll_pds = []

        image_FWHMs_pt = []
        image_SNRs_pt = []    
        MFBs_pt = []
        MFVs_pt = []
        Ps_normalised_pt = []
        B0s_pt = []
        norm_phot_resids_pt = []
        stamp_ids_pt = []
        ll_pts = []

        print('Image %d / %d' % (i, len(images)))
        print(fnames[i])

        # load corresponding flat
        date = fnames[i].split('_')[3]
        flat_index = np.where(np.array(master_flat_dates) == date)[0][0]
        print('Using flat:', master_flat_dates[flat_index])
        master_flat = master_flats[flat_index]

        # compute appropriate stamp and kernel size
        # the former will be used for PSF fitting photometry
        # a square (2 * FWHM) x (2 * FWHM) stamp is a reasonable choice
        fwhm = FWHMs[i]
        print('Image FWHM:', fwhm)
        ks = np.int(2*fwhm)
        if ks % 2 == 0: # must be odd!
            ks += 1
        print('Stamp and kernel size:', ks)
        
        # this is used for padding the image borders
        hwidth = np.int((ks - 1) / 2)

        # find integer shifts
        ys = shifts[:,1][np.where(shifts[:,0] == fnames[i])][0] 
        xs = shifts[:,2][np.where(shifts[:,0] == fnames[i])][0]

        # x and y need to be switched
        shiftxy = np.array([xs.astype(int), ys.astype(int)])

        # store values for photometric accuracy metrics
        # requires relative P for P_true
        photometric_scale_factors_pd = []
        F_measured_values_pd = []
        photometric_scale_factors_pt = []
        F_measured_values_pt = []    
        target_psf_objects = []
        pixel_uncertanties_list = []


        ## for each image, divide this into image and reference stamps centered on a star ##
        ## solve a kernel and background term for each, and perform psf fitting at the    ##
        ## position of the star in the difference image.

        ## deal with nans on first pass ##
        for j,pos in enumerate(positions[:, [0,1]]):

            try:

                # add shifts to positions so we get the right part of the reference and flat
                print('Source position in reference frame:', pos)
                print('Integer shift (x, y):', shiftxy)
                ref_stamp = make_stamp(ref, pos, cutout_size)
                image_stamp = make_stamp(image, pos + shiftxy, cutout_size)
                flat_stamp = make_stamp(master_flat, pos + shiftxy, cutout_size)
                ref_delta_stamp = make_stamp(ref_delta, pos, cutout_size)
                
                ################################################################################
                ######################## pyDANDIA ##############################################
                ## run pyDANDIA to build the bad pixel mask and pixel uncertanties so we compare
                ## like with like when calculating metrics
                mask = np.zeros(image_stamp.shape) # bad pixel mask
                for iters in range(0, 4):

                    if iters == 0:
                        shot_noise = image_stamp/(G*flat_stamp)
                    else:
                        shot_noise = model_pd/(G*flat_stamp)   

                    pixel_variances = excess_noise_factor*shot_noise
                    weights_stamp = 1./pixel_variances
                    
                    # mask 5-sigma outliers
                    if iters > 1:
                        # update mask on third and fourth iterations only
                        norm_resids = np.sqrt(weights_stamp)*(image_stamp - model_pd)
                        mask[np.where(np.abs(norm_resids)>5)] = 1
                        
                    # weight-out bad pixels
                    weights_stamp[np.where(mask == 1)] = 1e-99
                    print('\nMasked pixels (pyDANDIA):', np.sum(mask))
                    
                    print('\npyDANDIA solution, iter %d' % iters)
                    
                    # pad input images by half the kernel size
                    ext_ref = extend_image_hw(ref_stamp, ks)
                    ext_imag = extend_image_hw(image_stamp, ks)
                    ext_weights = extend_image_hw(weights_stamp, ks)
                    
                    U, b = construct_kernel_and_matrices(ks, ext_ref, ext_imag, ext_weights)
                    kernel_pd, B0_pd = lstsq_solution(ext_ref, ext_imag, U, b, ks)
                    print('P:', np.sum(kernel_pd))
                    print('B0:', B0_pd)

                    # compute the model - B08/pyDANDIA
                    ext_ref_stamp = extend_image_hw(ref_stamp, ks)
                    ext_model_pd = convolve2d(ext_ref_stamp, kernel_pd, mode='same') + B0_pd
                    model_pd = ext_model_pd[hwidth:ext_model_pd.shape[0]-hwidth,
                                            hwidth:ext_model_pd.shape[1]-hwidth]    


                    if iters == 3:
                        # calculate pixel uncertanties on the final, fourth iteration
                        shot_noise = model_pd/(G*flat_stamp)
                        pixel_variances = excess_noise_factor*shot_noise
                        pixel_uncertanties = np.sqrt(pixel_variances)


                #########################################################################
                #########################################################################
                
                ##################### PyTorchDIA ########################################
                # infer kernel via robust PyTorchDIA code
                print('\n(Robust) PyTorchDIA solution')
                SD_steps = 25000
                kernel_pt, B0_pt = PyTorchDIA_EMCCD.DIA(ref_stamp,
                                           image_stamp,
                                           flat_stamp,
                                           read_noise = 0.,
                                           ks = ks,
                                           lr_kernel = 1e-3,
                                           lr_B = 100,
                                           SD_steps = SD_steps,
                                           Newton_tol = 1e-6,
                                           poly_degree=0,
                                           fast=True,
                                           tol = 1e-9,
                                           max_iterations = SD_steps,
                                           fisher=False,
                                           show_convergence_plots=False)
                
                # compute the model - PyTorch
                ext_ref_stamp = extend_image_hw(ref_stamp, ks)
                ext_model_pt = convolve2d(ext_ref_stamp, kernel_pt, mode='same') + B0_pt
                model_pt = ext_model_pt[hwidth:ext_model_pt.shape[0]-hwidth,
                                        hwidth:ext_model_pt.shape[1]-hwidth] 
                
                # and variances for evaluating the Huber 'likelihood'
                shot_noise_pt = model_pt/(G*flat_stamp)
                pixel_variances_pt = excess_noise_factor*shot_noise_pt
                ############################################################################
                ############################################################################
                
                ### compute likelihood ratio only if there is no data rejection
                if np.sum(mask) == 0:
                    ll_pd = evaluate_gaussian_log_likelihood(image_stamp, model_pd, pixel_variances)
                    ll_pt = evaluate_huber_log_likelihood(image_stamp, model_pt, pixel_variances_pt, c=1.345)                
                    print('pyDANDIA (Gaussian) log-likelihood:', ll_pd)
                    print('PyTorch (Huber) log-likelihood:', ll_pt)
                    print('ll_pd - ll_pt:', ll_pd - ll_pt)
                    
                    ll_pds.append(ll_pd)
                    ll_pts.append(ll_pt)
                    
                else:
                    ll_pds.append(0)
                    ll_pts.append(0)                    

                # image models
                models = [model_pd, model_pt]


                #### PSF fitting photometry in difference image ####
                #### infer PSF - this is unweighted ###
                res = PyTorchDIA_Newton.DIA(ref_delta_stamp, image_stamp, flat_stamp,
                                              tot_im = Tot_ims[i], unweighted=True,
                                              iters=1, ks = ks, SD_steps = 0,
                                              tol = 1e-9, k=1e99)

                psf = res[0]

                #stop = input()
                psf /= np.sum(psf)

                for model in models:

                    if model is model_pd:
                        print('\nCalculating pyDANDIA metrics')
                    else:
                        print('\nCalculating PyTorchDIA metrics')

                    stamp_fit = make_fit_cutout(image_stamp - model, ks)
                    noise_stamp_fit = make_fit_cutout(pixel_uncertanties, ks)

                    ## psf fit
                    F_diff = torch.nn.Parameter(torch.ones(1), requires_grad=True)
                    const = torch.nn.Parameter(torch.ones(1), requires_grad=True)
                    print('Starting F_diff and const:', F_diff, const)

                    #print(model.shape, stamp_fit.shape, noise_stamp_fit.shape)

                    target_psf_object = torch.from_numpy(psf)
                    stamp_fit = torch.from_numpy(np.array(stamp_fit, dtype=np.float32))
                    noise_stamp_fit = torch.from_numpy(np.array(noise_stamp_fit, dtype=np.float32))

                    class log_likelihood(torch.nn.Module):

                        def forward(model, stamp, noise_stamp):
                            #print(stamp.size(), model.size(), noise_stamp.size())
                            loglikelihood = -0.5*(((stamp - model)/noise_stamp)**2).sum()
                            return -loglikelihood

                    optimizer = torch.optim.Adam([F_diff, const], lr=1000)

                    tol = 1e-10
                    losses = []
                    for step in range(0, 150000):
                        optimizer.zero_grad()
                        psf_model = F_diff*target_psf_object + const
                        loss = log_likelihood.forward(psf_model, stamp_fit, noise_stamp_fit)
                        losses.append(loss.item())
                        loss.backward()
                        optimizer.step()

                        if step>1 and abs((losses[-1] - losses[-2])/losses[-2]) < tol:
                            print('Converged')
                            break


                    print('Final F and const:', F_diff, const)
                    #plt.plot(losses)

                    ## convert tensors back to numpy arrays
                    F_diff = F_diff.detach().numpy()
                    const = const.detach().numpy()
                    target_psf_object = target_psf_object.detach().numpy()
                    stamp_fit = stamp_fit.detach().numpy()
                    noise_stamp_fit = noise_stamp_fit.detach().numpy()

                    '''
                    # plot the stamp
                    plt.figure()
                    plt.title('Difference image stamp')
                    plt.imshow(stamp_fit)
                    plt.colorbar()
                    plt.show()

                    # plot the reference stamp - to check aligned OK
                    plt.figure()
                    plt.title('Reference Image stamp')
                    plt.imshow(make_fit_cutout(ref_stamp, ks))
                    plt.colorbar()
                    plt.show()


                    # plot the image stamp - to check psf accurate!
                    plt.figure()
                    plt.title('Data Image stamp')
                    plt.imshow(make_fit_cutout(image_stamp, ks))
                    plt.colorbar()
                    plt.show()

                    # plot the psf model
                    plt.figure()
                    plt.title('Normalised PSF model')
                    plt.imshow(target_psf_object)
                    plt.colorbar()
                    plt.show()

                    # plot the residuals
                    prediction = F_diff*target_psf_object + const
                    residuals = prediction - stamp_fit
                    plt.figure()
                    plt.title('Residuals from PSF fit to difference image stamp')
                    plt.imshow(residuals)
                    plt.colorbar()
                    plt.show()
                    '''

                    if model is model_pd:

                        ## Measured flux of brightest star
                        F_measured_pd = F_diff / np.sum(kernel_pd)

                        # store values to cal
                        F_measured_values_pd.append(F_measured_pd)
                        photometric_scale_factors_pd.append(np.sum(kernel_pd))
                        target_psf_objects.append(target_psf_object)
                        pixel_uncertanties_list.append(noise_stamp_fit)

                        # stamp number
                        stamp_ids_pd.append(j)

                        # image SNR
                        sky = np.median(image_stamp)
                        SNR = np.sum(image_stamp - sky) / np.sqrt(np.sum(pixel_uncertanties**2))
                        image_SNRs_pd.append(SNR)
                        image_FWHMs_pd.append(FWHMs[i])

                        # apply bad pixel mask (same for both PyTorchDIA and pyDANDIA)
                        masked_model_pd = np.ma.array(model, mask=mask)
                        masked_image_stamp = np.ma.array(image_stamp, mask=mask)
                        masked_pixel_uncertanties = np.ma.array(pixel_uncertanties, mask=mask)

                        # metrics
                        MFB_pd, MFV_pd = metrics(masked_model_pd, masked_image_stamp,
                                                 masked_pixel_uncertanties, ks, mask)
                        
                        print('P:', np.sum(kernel_pd))
                        print('B0:', B0_pd)
                        print('MFB:', MFB_pd)
                        print('MFV:', MFV_pd)

                        B0s_pd.append(B0_pd)
                        MFBs_pd.append(MFB_pd)
                        MFVs_pd.append(MFV_pd)

                    else:

                        ## Measured flux of brightest star
                        F_measured = F_diff / np.sum(kernel_pt)

                        # store values to cal
                        F_measured_values_pt.append(F_measured)
                        photometric_scale_factors_pt.append(np.sum(kernel_pt))

                        # stamp number
                        stamp_ids_pt.append(j)

                        # image SNR
                        sky = np.median(image_stamp)
                        SNR = np.sum(image_stamp - sky) / np.sqrt(np.sum(pixel_uncertanties**2))
                        image_SNRs_pt.append(SNR)
                        image_FWHMs_pt.append(FWHMs[i])

                        # apply bad pixel mask (same for both PyTorchDIA and pyDANDIA)
                        masked_model_pt = np.ma.array(model, mask=mask)
                        masked_image_stamp = np.ma.array(image_stamp, mask=mask)
                        masked_pixel_uncertanties = np.ma.array(pixel_uncertanties, mask=mask)

                        # metrics
                        MFB_pt, MFV_pt = metrics(masked_model_pt, masked_image_stamp,
                                                 masked_pixel_uncertanties, ks, mask)
                        
                        print('P:', np.sum(kernel_pt))
                        print('B0:', B0_pt)
                        print('MFB:', MFB_pt)
                        print('MFV:', MFV_pt)

                        B0s_pt.append(B0_pt)
                        MFBs_pt.append(MFB_pt)
                        MFVs_pt.append(MFV_pt)

            except (ValueError, np.linalg.LinAlgError) as e:
                print('Skipping reference-image pair... target too close to border for this imge FWHM')


        # as with 'true' pixel uncertanties, use pyDANDIA to calculate
        # 'true' photometric scale factor
        P_true = np.median(photometric_scale_factors_pd)

        print('\npyDANDIA photometric residuals')
        for s in range(0, len(photometric_scale_factors_pd)):
            P_normalised = photometric_scale_factors_pd[s]/P_true
            Ps_normalised_pd.append(P_normalised)
            min_var = (1./P_true**2) * (np.sum((target_psf_objects[s]**2/pixel_uncertanties_list[s]**2)))**(-1)
            F_measured_sigma_min = F_measured_values_pd[s] / np.sqrt(min_var)
            norm_phot_resids_pd.append(F_measured_sigma_min)
            print('F_measure / sigma_min:', F_measured_sigma_min)

        # PyTorch - calculate theoretical min_var for each stamp
        print('\nPyTorchDIA photometric residuals')
        for s in range(0, len(photometric_scale_factors_pt)):
            P_normalised = photometric_scale_factors_pt[s]/P_true
            Ps_normalised_pt.append(P_normalised)
            min_var = (1./P_true**2) * (np.sum((target_psf_objects[s]**2/pixel_uncertanties_list[s]**2)))**(-1)
            F_measured_sigma_min = F_measured_values_pt[s] / np.sqrt(min_var)
            norm_phot_resids_pt.append(F_measured_sigma_min)
            print('F_measure / sigma_min:', F_measured_sigma_min)



        out_pd = np.vstack((MFBs_pd, MFVs_pd, Ps_normalised_pd, np.array(norm_phot_resids_pd).flatten(),
                         image_SNRs_pd, image_FWHMs_pd, stamp_ids_pd, ll_pds)).T

        out_pt = np.vstack((MFBs_pt, MFVs_pt, Ps_normalised_pt, np.array(norm_phot_resids_pt).flatten(),
                         image_SNRs_pt, image_FWHMs_pt, stamp_ids_pt, ll_pts)).T

        
        
        #path = '/media/james/Seagate_Expansion_Drive#2'
        path = os.getcwd()

        filename = os.path.join(path, 'pyDANDIA_OGLE_BLG101_December2020_TestRun.txt')
        with open(filename, 'a') as f:
            np.savetxt(f, out_pd)
        
        filename = os.path.join(path, 'PyTorchDIA_OGLE_BLG101_December_2020_TestRun.txt')
        with open(filename, 'a') as f:
            np.savetxt(f, out_pt)
        

print(len(MFBs), len(MFVs), len(Ps_normalised), len(norm_phot_resids))
print('Total time:', time.time() - start_time)

Image 0 / 250
coll_OGLE-III-BLG101_Llr_2019-07-26_00010.fits
Using flat: 2019-07-26
Image FWHM: 10.05790566350711
Stamp and kernel size: 21
Source position in reference frame: [219.94513057  52.63611808]
Integer shift (x, y): [13 -2]

Masked pixels (pyDANDIA): 0.0

pyDANDIA solution, iter 0


KeyboardInterrupt: 